In [ ]:
import fastai
from fastai.vision.all import * 
!pip install -Uqq fastbook

from fastbook import *

In [ ]:
def f(x):
    return 3*x**2 + 2*x + 1
plot_function(f)

In [ ]:
# plot_function??

In [ ]:
def quad(a,b,c,x):
    return a*x**2 + b*x + c

In [ ]:
quad(3,2,1,1.5)

In [ ]:
from functools import partial

def draw_quad(a,b,c):
    return partial(quad, a,b,c)    

Partial is done basically to make coefficients a,b and c same while varying the value of x. What this does is, it helps to produce several results on that funciton with coefficients unchanged

In [ ]:
f = draw_quad(3,2,1)
f(1.8)

In [ ]:
plot_function(f)  # f = 3*x**2 + 2*x + 1 

Partial and Plot_function pretty much go hand in hand. Inorder to implement plot_function, we need to make clever use of partial 

In [ ]:
from numpy.random import normal,seed,uniform
np.random.seed(42)

def noise(x, scale):
    return normal(scale = scale, size = x.shape)

def add_noise(x, mult, add):
    return x * (1 + noise(x, mult)) + noise(x, add)
    print(x * (1 + noise(x, mult)) + noise(x, add))


In [ ]:
x = torch.linspace(-2, 2, steps = 20)
y = add_noise(f(x), 0.3, 1.5)          # added the noise
plt.scatter(x,y)

In [ ]:
y

In [ ]:
from matplotlib.pyplot import *
import matplotlib.pyplot as plt
from ipywidgets import interact
@interact(a = 1.5, b = 1.5, c = 1.5)
def plot_quad(a, b, c):
    plot_function(draw_quad(a, b, c))
    plt.scatter(x,y)

In [ ]:
loss = mae(f(x),y)
loss

In [ ]:
import numpy as np
from matplotlib.pyplot import *
import matplotlib.pyplot as plt
from ipywidgets import interact
@interact(a = 1.5, b = 1.5, c = 1.5)
def plot_quad(a, b, c):
    f = draw_quad(a,b,c)
    loss = mae(f(x),y).numpy()            #converting tensorbase into numpy arrays 
    plot_function(f, title=f"MSE: {loss:.2f}")
    plt.scatter(x,y)

In [ ]:
def quad_mse(params):
    f = draw_quad(*params)
    return mae(f(x),y)

In [ ]:
quad_mse([1,2,3])

In [ ]:
abc = torch.tensor([1.5, 1.5, 1.5])
abc.requires_grad_()

In [ ]:
loss = quad_mse(abc)
loss

In [ ]:
loss.backward()
abc.grad

In [ ]:
abc = torch.tensor([1.5, 1.5, 1.5])
abc.requires_grad_()

for i in range(16):
    loss = quad_mse(abc)
    loss.backward()
    with torch.no_grad():
        abc -= abc.grad * 0.01
        loss = quad_mse(abc)
        numpy_loss = loss.numpy()
    print(f'step = {i}; loss = {numpy_loss:.2f}')

"with torch.no_grad()" disables the calculation of gradients for any operations inside that context manager. We have to do that, because abc -= abc.grad*0.01 isn't actually part of our quadratic model, so we don't want derivitives to include that calculation.

We should always only calculate the gradient of the loss. Here, we only want to update the parameters. The program explicitly uses it here inorder not to update the gradients when it is updating the weights as that could affect the back propagation.

In [ ]:
def rectified_linear(m,b,x):
    y = m*x + b
    return torch.clip(y,0.)

In [ ]:
# plot_function??

In [ ]:
plot_function(partial(rectified_linear, 1,1))

In [ ]:
@interact(m = 1, b = 1)
def plot_relu(m, b):
    plot_function(partial(rectified_linear ,m,b))
    

In [ ]:
def double_relu(m1,b1,m2,b2,m3,b3,x):
    return rectified_linear(m1,b1,x) + rectified_linear(m2,b2,x) + rectified_linear(m3,b3,x)

In [ ]:
@interact(m1=1,b1=-1.5,m2=-1.5,b2=1.5,m3 = 1.5, b3 = 1.5)
def plot_triple_relu(m1, b1, m2, b2,m3,b3):
    plot_function(partial(double_relu, m1,b1,m2,b2,m3,b3))

As we can see, by just adding two reLUs, we can actually predict some complex data structures thereby matching some underlying patterns of the queries. Hence, we minimize the loss using gradient descent and again update the parameters. Several additions of reLU make a wigly like structure